In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Getting started to chat with the Gemini Pro model



<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_chat.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_chat.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_chat.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>


## Overview

This notebook demonstrates how to send chat prompts to the Gemini Pro model (`gemini-pro`) by using the Vertex AI SDK for Python and LangChain. Gemini Pro supports prompts with text-only input, including natural language tasks, multi-turn text and code chat, and code generation. It can output text and code.

Learn more about [Sending chat prompt requests (Gemini)](https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/send-chat-prompts-gemini).

### Objectives

In this tutorial, you learn how to send chat prompts to the Gemini Pro model (`gemini-pro`) using the Vertex AI SDK for Python and LangChain.

You will complete the following tasks:

- Sending chat prompts using Vertex AI SDK for Python
- Sending chat prompts using LangChain

### Costs
This tutorial uses billable components of Google Cloud:

- Vertex AI

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Getting Started

### Install libraries


In [1]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 langchain-google-vertexai \
                                 langchain

### Restart current runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

The restart might take a minute or longer. After its restarted, continue to the next step.

In [2]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ Wait for the kernel to finish restarting before you continue. ⚠️</b>
</div>

### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the cell below to authenticate your environment.

This step is not required if you are using [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench).

In [2]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:

    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

### Define Google Cloud project information (Colab only)

If you are running this notebook on Google Colab, you need to define Google Cloud project information to be used. In the following cell, you will define the information, import the Vertex AI package, and initialize it.

This step is not required if you are using [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench).

In [3]:
if "google.colab" in sys.modules:

    # Define project information
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
    LOCATION = "us-central1"  # @param {type:"string"}

    # Initialize Vertex AI
    import vertexai

    vertexai.init(project=PROJECT_ID, location=LOCATION)

### Import libraries

In [6]:
from IPython.display import Markdown
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import (ChatPromptTemplate, HumanMessagePromptTemplate,
                               MessagesPlaceholder,
                               SystemMessagePromptTemplate)
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_google_vertexai import (ChatVertexAI, HarmBlockThreshold,
                                       HarmCategory)
from vertexai.preview.generative_models import Content, GenerativeModel, Part

## Sending chat prompts using Vertex AI SDK for Python

### Load the Gemini Pro model

Gemini Pro supports text and code generation from a text prompt.

In [7]:
model = GenerativeModel("gemini-pro")

### Start a chat session

You start a stateful chat session and then send chat prompts with configuration parameters including generation configurations and safety settings.

In [8]:
chat = model.start_chat()

response = chat.send_message(
    """You are an astronomer, knowledgeable about the solar system.
How many moons does Mars have? Tell me some fun facts about them.
"""
)

print(response.text)

Mars has two moons, Phobos and Deimos.

**Phobos:**
- Phobos is the larger and closer of the two moons, with a diameter of about 22.2 kilometers.
- It orbits Mars three times a day, completing one orbit in just 7 hours and 39 minutes. This makes it the fastest-orbiting moon in the solar system.
- Phobos is heavily cratered, and its surface is covered in fine-grained dust.
- It is thought to be a captured asteroid rather than a moon formed from the same material as Mars.

**Deimos:**
- Deimos is the smaller and farther of the two moons, with a diameter of about 12.6 kilometers.
- It orbits Mars once every 30 hours and 18 minutes.
- Deimos is less cratered than Phobos and has a smoother surface.
- Its shape is irregular, and it is thought to be a captured asteroid or a fragment of a larger moon that was disrupted long ago.

Fun facts about the moons of Mars:

- The names Phobos and Deimos come from the Greek words for "fear" and "terror," respectively. They were the sons of the Greek god

You can use `Markdown` to display the generated text.

In [9]:
Markdown(response.text)

Mars has two moons, Phobos and Deimos.

**Phobos:**
- Phobos is the larger and closer of the two moons, with a diameter of about 22.2 kilometers.
- It orbits Mars three times a day, completing one orbit in just 7 hours and 39 minutes. This makes it the fastest-orbiting moon in the solar system.
- Phobos is heavily cratered, and its surface is covered in fine-grained dust.
- It is thought to be a captured asteroid rather than a moon formed from the same material as Mars.

**Deimos:**
- Deimos is the smaller and farther of the two moons, with a diameter of about 12.6 kilometers.
- It orbits Mars once every 30 hours and 18 minutes.
- Deimos is less cratered than Phobos and has a smoother surface.
- Its shape is irregular, and it is thought to be a captured asteroid or a fragment of a larger moon that was disrupted long ago.

Fun facts about the moons of Mars:

- The names Phobos and Deimos come from the Greek words for "fear" and "terror," respectively. They were the sons of the Greek god Ares, who is the Roman equivalent of Mars.
- Phobos is the only moon in the solar system known to have a retrograde orbit, meaning that it orbits Mars in the opposite direction of the planet's rotation.
- Deimos is the outermost moon in the solar system.
- Both Phobos and Deimos are thought to be relatively young, with ages of only a few hundred million years.
- The moons of Mars are not visible from Earth with the naked eye, even with a telescope.

You can check out the metadata of the response including the `safety_ratings` and `usage_metedata`.

In [10]:
print(response)

candidates {
  content {
    role: "model"
    parts {
      text: "Mars has two moons, Phobos and Deimos.\n\n**Phobos:**\n- Phobos is the larger and closer of the two moons, with a diameter of about 22.2 kilometers.\n- It orbits Mars three times a day, completing one orbit in just 7 hours and 39 minutes. This makes it the fastest-orbiting moon in the solar system.\n- Phobos is heavily cratered, and its surface is covered in fine-grained dust.\n- It is thought to be a captured asteroid rather than a moon formed from the same material as Mars.\n\n**Deimos:**\n- Deimos is the smaller and farther of the two moons, with a diameter of about 12.6 kilometers.\n- It orbits Mars once every 30 hours and 18 minutes.\n- Deimos is less cratered than Phobos and has a smoother surface.\n- Its shape is irregular, and it is thought to be a captured asteroid or a fragment of a larger moon that was disrupted long ago.\n\nFun facts about the moons of Mars:\n\n- The names Phobos and Deimos come from the Gr

You can retrieve the history of the chat session.

In [11]:
print(chat.history)

[role: "user"
parts {
  text: "You are an astronomer, knowledgeable about the solar system.\nHow many moons does Mars have? Tell me some fun facts about them.\n"
}
, role: "model"
parts {
  text: "Mars has two moons, Phobos and Deimos.\n\n**Phobos:**\n- Phobos is the larger and closer of the two moons, with a diameter of about 22.2 kilometers.\n- It orbits Mars three times a day, completing one orbit in just 7 hours and 39 minutes. This makes it the fastest-orbiting moon in the solar system.\n- Phobos is heavily cratered, and its surface is covered in fine-grained dust.\n- It is thought to be a captured asteroid rather than a moon formed from the same material as Mars.\n\n**Deimos:**\n- Deimos is the smaller and farther of the two moons, with a diameter of about 12.6 kilometers.\n- It orbits Mars once every 30 hours and 18 minutes.\n- Deimos is less cratered than Phobos and has a smoother surface.\n- Its shape is irregular, and it is thought to be a captured asteroid or a fragment of a

### Code chat

Gemini Pro also supports code generation from a text prompt.

In [12]:
code_chat = model.start_chat()

response = code_chat.send_message(
    "Write a function that checks if a year is a leap year"
)

print(response.text)

```python
def is_leap_year(year):
  """
  Checks if a year is a leap year.

  Args:
    year: The year to check.

  Returns:
    True if the year is a leap year, False otherwise.
  """

  if year % 4 != 0:
    return False

  if year % 100 == 0 and year % 400 != 0:
    return False

  return True
```


You can generate unit tests to test the function in this multi-turn chat.

In [13]:
response = code_chat.send_message("Write a unit test of the generated function")

print(response.text)

```python
import unittest

class LeapYearTest(unittest.TestCase):

    def test_is_leap_year_1900(self):
        self.assertFalse(is_leap_year(1900))

    def test_is_leap_year_2000(self):
        self.assertTrue(is_leap_year(2000))

    def test_is_leap_year_2004(self):
        self.assertTrue(is_leap_year(2004))

    def test_is_leap_year_2008(self):
        self.assertTrue(is_leap_year(2008))

    def test_is_leap_year_2012(self):
        self.assertTrue(is_leap_year(2012))

    def test_is_leap_year_2016(self):
        self.assertTrue(is_leap_year(2016))

    def test_is_leap_year_2020(self):
        self.assertTrue(is_leap_year(2020))

    def test_is_leap_year_2024(self):
        self.assertTrue(is_leap_year(2024))

    def test_is_leap_year_2028(self):
        self.assertTrue(is_leap_year(2028))

    def test_is_leap_year_2032(self):
        self.assertTrue(is_leap_year(2032))

    def test_is_leap_year_2036(self):
        self.assertTrue(is_leap_year(2036))

    def test_is_lea

### Add chat history

You can add chat history to a chat by adding messages from role `user` and `model` alternately. System messages can be set in the first part for the first message.

In [14]:
chat2 = model.start_chat(
    history=[
        Content(
            role="user",
            parts=[
                Part.from_text(
                    """
    My name is Ned. You are my personal assistant. My favorite movies are Lord of the Rings and Hobbit.
    Who do you work for?
    """
                )
            ],
        ),
        Content(role="model", parts=[Part.from_text("I work for Ned.")]),
        Content(role="user", parts=[Part.from_text("What do I like?")]),
        Content(role="model", parts=[Part.from_text("Ned likes watching movies.")]),
    ]
)

response = chat2.send_message("Are my favorite movies based on a book series?")
Markdown(response.text)

Yes, Ned's favorite movies, Lord of the Rings and Hobbit, are both based on a book series.

* The Lord of the Rings is based on the novel of the same name by J.R.R. Tolkien.
* The Hobbit is based on the novel of the same name by J.R.R. Tolkien.

In [15]:
response = chat2.send_message("When were these books published?")
Markdown(response.text)

* **The Hobbit:** 1937
* **The Lord of the Rings:** 1954–1955

The Lord of the Rings is actually a sequel to The Hobbit, and both are set in the same fictional universe of Middle-earth.

## Sending chat prompts using LangChain

The Vertex AI Gemini API is integrated with the LangChain Python SDK, making it convenient to build applications on top of Gemini models.

### Start a chat session

You can start a chat by sending chat prompts to the Gemini Pro model directly. Gemini Pro doesn’t support `SystemMessage` at the moment, but `SystemMessage` can be added to the first human message by setting the `convert_system_message_to_human` to `True`.

In [16]:
system_message = "You are a helpful assistant who translates English to French."
human_message = "Translate this sentence from English to French. I love programming."

messages = [SystemMessage(content=system_message), HumanMessage(content=human_message)]

chat = ChatVertexAI(
    model_name="gemini-pro",
    convert_system_message_to_human=True,
    safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE
    },
)

result = chat.generate([messages])
print(result.generations[0][0].text)

J'adore la programmation.


You can check out the metadata of the generated content.

In [17]:
print(result.generations[0][0].generation_info)

{'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE'}]}


### Use a chat chain with chat prompt template

In [18]:
system_message = "You are a helpful assistant who translates English to French."
human_message = "Translate this sentence from English to French. I love programming."

messages = [SystemMessage(content=system_message), HumanMessage(content=human_message)]
prompt = ChatPromptTemplate.from_messages(messages)

chat = ChatVertexAI(
    model_name="gemini-pro",
    convert_system_message_to_human=True,
    safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE
    },
)

chain = prompt | chat
chain.invoke({})

AIMessage(content="J'aime la programmation.")

### Use a conversation chain

You also can wrap up a chat in `ConversationChain`, which has built-in memory for remembering past user inputs and model outputs.

In [19]:
model = ChatVertexAI(
    model_name="gemini-pro",
    convert_system_message_to_human=True,
    safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE
    },
)

prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a helpful assistant who is good at language translation."
        ),
        MessagesPlaceholder(variable_name="history"),
        HumanMessagePromptTemplate.from_template("{input}"),
    ]
)

memory = ConversationBufferMemory(memory_key="history", return_messages=True)
conversation = ConversationChain(llm=model, prompt=prompt, verbose=True, memory=memory)

conversation.run(
    input="Translate this sentence from English to French. I love programming."
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new ConversationChain chain...
Prompt after formatting:
System: You are a helpful assistant who is good at language translation.
Human: Translate this sentence from English to French. I love programming.

> Finished chain.


"J'aime la programmation."

In [20]:
conversation.run("Translate it to Spanish")



> Entering new ConversationChain chain...
Prompt after formatting:
System: You are a helpful assistant who is good at language translation.
Human: Translate this sentence from English to French. I love programming.
AI: J'aime la programmation.
Human: Translate it to Spanish

> Finished chain.


'Me encanta la programación.'